In [36]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic_2d
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
from math import pi

In [37]:
chicago = "shot log CHI.csv"

chicago_df = pd.read_csv(chicago)
chicago_df

,self previous shot,player position,home game,location x,opponent previous shot,home team,shot type,points,away team,location y,time,date,shoot player,time from last shot,quarter,current shot outcome
0,NaN,SG,Yes,239.0,NaN,CHI,Jump Shot,2,BOS,169.0,0:17,2016-10-27,Dwyane Wade,NaN,1,SCORED
1,SCORED,PF,Yes,64.0,MISSED,CHI,Hook Shot,2,BOS,215.0,1:08,2016-10-27,Taj Gibson,51.0,1,MISSED
2,MISSED,PG,Yes,67.0,MISSED,CHI,Driving Layup,2,BOS,259.0,1:15,2016-10-27,Rajon Rondo,7.0,1,MISSED
3,MISSED,C,Yes,102.0,MISSED,CHI,Hook Shot,2,BOS,243.0,1:27,2016-10-27,Robin Lopez,12.0,1,MISSED
4,MISSED,SG,Yes,241.0,SCORED,CHI,Jump Shot,2,BOS,210.0,1:57,2016-10-27,Dwyane Wade,30.0,1,MISSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED


In [38]:
shot_df = chicago_df.rename(columns={"location x": "LOC_X", "location y": "LOC_Y", "player position": "position", "shoot player": "shooter"})
shot_df

,self previous shot,position,home game,LOC_X,opponent previous shot,home team,shot type,points,away team,LOC_Y,time,date,shooter,time from last shot,quarter,current shot outcome
0,NaN,SG,Yes,239.0,NaN,CHI,Jump Shot,2,BOS,169.0,0:17,2016-10-27,Dwyane Wade,NaN,1,SCORED
1,SCORED,PF,Yes,64.0,MISSED,CHI,Hook Shot,2,BOS,215.0,1:08,2016-10-27,Taj Gibson,51.0,1,MISSED
2,MISSED,PG,Yes,67.0,MISSED,CHI,Driving Layup,2,BOS,259.0,1:15,2016-10-27,Rajon Rondo,7.0,1,MISSED
3,MISSED,C,Yes,102.0,MISSED,CHI,Hook Shot,2,BOS,243.0,1:27,2016-10-27,Robin Lopez,12.0,1,MISSED
4,MISSED,SG,Yes,241.0,SCORED,CHI,Jump Shot,2,BOS,210.0,1:57,2016-10-27,Dwyane Wade,30.0,1,MISSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED


In [39]:
# Get names of indexes for which column Age has value 30
farCourt = shot_df[ shot_df['LOC_X'] < 470 ].index
# Delete these row indexes from dataFrame
shot_df.drop(farCourt , inplace=True)

In [40]:
shot_df

,self previous shot,position,home game,LOC_X,opponent previous shot,home team,shot type,points,away team,LOC_Y,time,date,shooter,time from last shot,quarter,current shot outcome
47,SCORED,PF,Yes,863.0,SCORED,CHI,Driving Layup,2,BOS,251.0,0:33,2016-10-27,Taj Gibson,NaN,3,MISSED
48,MISSED,C,Yes,887.0,SCORED,CHI,Tip Layup Shot,2,BOS,241.0,0:35,2016-10-27,Robin Lopez,2.0,3,SCORED
49,SCORED,PG,Yes,886.0,SCORED,CHI,Pullup Jump Shot,2,BOS,413.0,1:22,2016-10-27,Rajon Rondo,47.0,3,MISSED
50,MISSED,PF,Yes,706.0,SCORED,CHI,Jump Shot,2,BOS,174.0,1:42,2016-10-27,Taj Gibson,20.0,3,SCORED
51,SCORED,SG,Yes,890.0,MISSED,CHI,Jump Shot,2,BOS,433.0,2:05,2016-10-27,Dwyane Wade,23.0,3,MISSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED


In [41]:
shot_df['LOC_X2'] = shot_df.apply(lambda row: (row.LOC_Y - 500) * (-1), axis = 1) 

new_y_df = shot_df

In [42]:
new_y_df

,self previous shot,position,home game,LOC_X,opponent previous shot,home team,shot type,points,away team,LOC_Y,time,date,shooter,time from last shot,quarter,current shot outcome,LOC_X2
47,SCORED,PF,Yes,863.0,SCORED,CHI,Driving Layup,2,BOS,251.0,0:33,2016-10-27,Taj Gibson,NaN,3,MISSED,249.0
48,MISSED,C,Yes,887.0,SCORED,CHI,Tip Layup Shot,2,BOS,241.0,0:35,2016-10-27,Robin Lopez,2.0,3,SCORED,259.0
49,SCORED,PG,Yes,886.0,SCORED,CHI,Pullup Jump Shot,2,BOS,413.0,1:22,2016-10-27,Rajon Rondo,47.0,3,MISSED,87.0
50,MISSED,PF,Yes,706.0,SCORED,CHI,Jump Shot,2,BOS,174.0,1:42,2016-10-27,Taj Gibson,20.0,3,SCORED,326.0
51,SCORED,SG,Yes,890.0,MISSED,CHI,Jump Shot,2,BOS,433.0,2:05,2016-10-27,Dwyane Wade,23.0,3,MISSED,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED,288.0
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED,387.0
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED,388.0
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED,401.0


In [43]:
new_y_df['distance_from_half'] = new_y_df.apply(lambda row: (row.LOC_X - 470) * 2, axis = 1) 

In [44]:
new_y_df

,self previous shot,position,home game,LOC_X,opponent previous shot,home team,shot type,points,away team,LOC_Y,time,date,shooter,time from last shot,quarter,current shot outcome,LOC_X2,distance_from_half
47,SCORED,PF,Yes,863.0,SCORED,CHI,Driving Layup,2,BOS,251.0,0:33,2016-10-27,Taj Gibson,NaN,3,MISSED,249.0,786.0
48,MISSED,C,Yes,887.0,SCORED,CHI,Tip Layup Shot,2,BOS,241.0,0:35,2016-10-27,Robin Lopez,2.0,3,SCORED,259.0,834.0
49,SCORED,PG,Yes,886.0,SCORED,CHI,Pullup Jump Shot,2,BOS,413.0,1:22,2016-10-27,Rajon Rondo,47.0,3,MISSED,87.0,832.0
50,MISSED,PF,Yes,706.0,SCORED,CHI,Jump Shot,2,BOS,174.0,1:42,2016-10-27,Taj Gibson,20.0,3,SCORED,326.0,472.0
51,SCORED,SG,Yes,890.0,MISSED,CHI,Jump Shot,2,BOS,433.0,2:05,2016-10-27,Dwyane Wade,23.0,3,MISSED,67.0,840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED,288.0,340.0
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED,387.0,410.0
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED,388.0,756.0
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED,401.0,442.0


In [45]:
new_y_df['LOC_Y2'] = new_y_df.LOC_X - new_y_df.distance_from_half
new_y_df

,self previous shot,position,home game,LOC_X,opponent previous shot,home team,shot type,points,away team,LOC_Y,time,date,shooter,time from last shot,quarter,current shot outcome,LOC_X2,distance_from_half,LOC_Y2
47,SCORED,PF,Yes,863.0,SCORED,CHI,Driving Layup,2,BOS,251.0,0:33,2016-10-27,Taj Gibson,NaN,3,MISSED,249.0,786.0,77.0
48,MISSED,C,Yes,887.0,SCORED,CHI,Tip Layup Shot,2,BOS,241.0,0:35,2016-10-27,Robin Lopez,2.0,3,SCORED,259.0,834.0,53.0
49,SCORED,PG,Yes,886.0,SCORED,CHI,Pullup Jump Shot,2,BOS,413.0,1:22,2016-10-27,Rajon Rondo,47.0,3,MISSED,87.0,832.0,54.0
50,MISSED,PF,Yes,706.0,SCORED,CHI,Jump Shot,2,BOS,174.0,1:42,2016-10-27,Taj Gibson,20.0,3,SCORED,326.0,472.0,234.0
51,SCORED,SG,Yes,890.0,MISSED,CHI,Jump Shot,2,BOS,433.0,2:05,2016-10-27,Dwyane Wade,23.0,3,MISSED,67.0,840.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7134,SCORED,G,Yes,640.0,MISSED,CHI,Jump Shot,3,BRO,212.0,9:42,2017-04-12,Denzel Valentine,27.0,4,SCORED,288.0,340.0,300.0
7135,SCORED,G,Yes,675.0,MISSED,CHI,Jump Shot,3,BRO,113.0,10:13,2017-04-12,Denzel Valentine,31.0,4,MISSED,387.0,410.0,265.0
7136,MISSED,SG,Yes,848.0,SCORED,CHI,Jump Shot,2,BRO,112.0,10:39,2017-04-12,Anthony Morrow,26.0,4,SCORED,388.0,756.0,92.0
7137,SCORED,PF,Yes,691.0,MISSED,CHI,Jump Shot,3,BRO,99.0,11:10,2017-04-12,Bobby Portis,31.0,4,SCORED,401.0,442.0,249.0
